In [ ]:
pip install trimesh

In [ ]:
import os
import random
import glob
import trimesh
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv3D, Dense ,Dropout, Flatten, Activation, MaxPooling3D, Input

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    print("reading from cloud...")
    path = "/content/drive/MyDrive/Colab Notebooks/MPPE/stls_opt/"
except:
    import pathlib
    print("reading from disk")
    path = str(pathlib.Path().resolve())+"/dataset/stls_opt/"

In [ ]:
mesh = trimesh.load(path+"63_8475_10313_17790_28845.stl")

In [ ]:
def augment(points):
    # jitter points
    points += tf.random.uniform(points.shape, -0.05, 0.05, dtype=tf.float32)
    # shuffle points
    points = tf.random.shuffle(points)
    return points


In [ ]:
mv =np.array(mesh.volume)

In [ ]:
#a = np.zeros(0)
a = np.append(a,mesh.volume+mesh.volume*2)
print(a)

In [ ]:
n_samples = 10000
n_aug_samples = 1
X_train = np.zeros([0,n_samples,3])
X_train_volume = np.zeros(0)
y_train = np.zeros(0)
for filename in os.listdir(path):
    if filename.endswith(".stl"):
        print("appending", filename)
        mesh = trimesh.load(path+filename)
        points = mesh.sample(n_samples)
        
        for n in range(n_aug_samples):

            mv = mesh.volume
            mv = mv + mv*random.randint(-5,5)/10000
            X_train_volume = np.append(X_train_volume,mv)

            points = augment(points)
            X_train = np.append(X_train,points[np.newaxis,...],axis=0)
            y_train = np.append(y_train,filename.split("_")[3])
    

In [ ]:
points = mesh.sample(5000)
#points = augment(points)
for points in X_train:
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111,projection="3d")
    ax.scatter(points[:, 0], points[:, 2], points[:, 1])
    ax.set_axis_off()
    plt.show()

In [ ]:
X_train_volume.shape

In [ ]:
X_train.shape

In [ ]:
#Load model (LSTM_3D is the one that worked the best), set learning rate and number of classes (final layer length), then train the data (change epoch for more training cycles)
#model = CNN_3D_REG(X_train,learning_rate= 0.00002)
#print(model.summary())
# model.fit(X_train, y_train, batch_size = 8, validation_split = 0.2, epochs = 15, verbose = 2)
model.fit(X_train, y_train, batch_size = 8,validation_split = 0.2, epochs = 10, verbose = 2)
#esults = model.evaluate(X_test, y_test, verbose = 2)
print('Test accuracy: ', results[1])

In [ ]:
X_train_volume

In [ ]:
input1 = keras.Input(shape=(X_train.shape[1:]))
input2 = keras.Input(shape=(1))

y = Dense(1)(input2)

x = tnet(input1, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
#x = layers.Dropout(0.3)(x)

x = layers.Concatenate()([x,y])

dense = Dense(2000, activation = 'relu')(x)
dense = Dense(300, activation = 'relu')(dense)
dense = Dense(150, activation = 'relu')(dense)
dense = Dense(20, activation = 'relu')(dense)
dense = Dense(16, activation = 'relu')(dense)
    # final layer with 10 neurons to classify the instances
output = Dense(1, activation = 'linear')(dense)
    
#outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=[input1,input2], outputs=output, name="pointnet")
#model.summary()

In [ ]:
def augment(points):
    # jitter points
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    # shuffle points
    points = tf.random.shuffle(points)
    return points

#fuctions to build models 
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

#function to create T-net layers
def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [ ]:
y_train = np.float64(y_train)
y_train_max = y_train.max()
model.compile(
    loss="MSE",
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy"],
)

model.fit([X_train, X_train_volume], y_train/y_train_max, batch_size = 8, epochs = 5, verbose = 2,validation_split=0.2)


In [ ]:
y_train_max = y_train.max()
preds = model.predict(X_train)#*y_train_max

In [ ]:
preds.flatten()